In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Drop passengerID and Name, as these data may not necessary for the model

def drop(df):
    df.drop(['PassengerId', 'Name'], axis=1, inplace=True)

In [3]:
# fill null record
def fill(df):
    df['Cabin_num'] = df['Cabin_num'].fillna(value='0')
    df[['CryoSleep', 'VIP']] = df[['CryoSleep', 'VIP']].fillna(value=False)
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['RoomService'] = df['RoomService'].fillna(df['RoomService'].mean())
    df['FoodCourt'] = df['FoodCourt'].fillna(df['FoodCourt'].mean())
    df['ShoppingMall'] = df['ShoppingMall'].fillna(df['ShoppingMall'].mean())
    df['Spa'] = df['Spa'].fillna(df['Spa'].mean())
    df['VRDeck'] = df['VRDeck'].fillna(df['VRDeck'].mean())

In [4]:
# split cabin into 3 column, cabin_deck, cabin_num, cabin_side
def splitCabin(df):
    temp = pd.DataFrame()
    temp[['Cabin_deck', 'Cabin_num', 'Cabin_side']] = df['Cabin'].str.split(pat='/', expand=True)
    df.insert(2,'Cabin_side' ,temp['Cabin_side'])
    df.insert(2,'Cabin_num' ,temp['Cabin_num'])
    df.insert(2,'Cabin_deck' ,temp['Cabin_deck'])
    df.drop('Cabin', axis=1, inplace=True)  


In [5]:
# convert some data type into int and bool
def typeConvert(df):
    df['CryoSleep'] = df['CryoSleep'].astype(bool)
    df['Cabin_num'] = df['Cabin_num'].astype(int)
    df['VIP'] = df['VIP'].astype(bool)

In [6]:
# apply one-hot encoding to non-numberic data
def oneHot(df, dataset):
    one_hot= pd.DataFrame()
    one_hot = pd.get_dummies(df)

    # reorder the dataframe
    if dataset == 'train':
        one_hot = one_hot[['HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars',
                               'CryoSleep', 'Cabin_deck_A', 'Cabin_deck_B', 'Cabin_deck_C', 'Cabin_deck_D',
                               'Cabin_deck_E', 'Cabin_deck_F', 'Cabin_deck_G', 'Cabin_deck_T',
                               'Cabin_num', 'Cabin_side_P', 'Cabin_side_S', 'Destination_55 Cancri e',
                               'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'Age', 'VIP',
                               'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Transported']]
    elif dataset == 'test':
        one_hot = one_hot[['HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars',
                               'CryoSleep', 'Cabin_deck_A', 'Cabin_deck_B', 'Cabin_deck_C', 'Cabin_deck_D',
                               'Cabin_deck_E', 'Cabin_deck_F', 'Cabin_deck_G', 'Cabin_deck_T',
                               'Cabin_num', 'Cabin_side_P', 'Cabin_side_S', 'Destination_55 Cancri e',
                               'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'Age', 'VIP',
                               'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']]

    return one_hot

In [7]:
# normalize the data
def norm(df):
    cabin_num = df['Cabin_num']
    age = df['Age']
    temp = df.iloc[:, 20:25]

    scaler = MinMaxScaler()
    cabin_num = scaler.fit_transform(pd.DataFrame(cabin_num))
    age = scaler.fit_transform(pd.DataFrame(age))
    temp = scaler.fit_transform(temp)

    normalized = df
    normalized['Cabin_num'] = pd.DataFrame(cabin_num)
    normalized['Age'] = pd.DataFrame(age)
    normalized.iloc[:, 20:25] = temp

    return normalized


In [8]:
# insert dataset
train_path = '../spaceship-titanic_rawData/spaceship_train.csv'
test_path = '../spaceship-titanic_rawData/spaceship_test.csv'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

train_data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [9]:
# drop
drop(train_data)
drop(test_data)

train_data

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False
8689,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False
8690,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True
8691,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False


In [10]:
# split cabin
splitCabin(train_data)
splitCabin(test_data)

test_data.isna().sum()

HomePlanet       87
CryoSleep        93
Cabin_deck      100
Cabin_num       100
Cabin_side      100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
dtype: int64

In [11]:
# fill null record
# the non-numberic feature do not need to fill null value, they will apply one-hot encoding later on
fill(train_data)
fill(test_data)

train_data.isna().sum()


HomePlanet      201
CryoSleep         0
Cabin_deck      199
Cabin_num         0
Cabin_side      199
Destination     182
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
dtype: int64

In [12]:
test_data.isna().sum()

HomePlanet       87
CryoSleep         0
Cabin_deck      100
Cabin_num         0
Cabin_side      100
Destination      92
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
dtype: int64

In [13]:
# datatype convertion
typeConvert(train_data)
typeConvert(test_data)

train_data.dtypes

HomePlanet       object
CryoSleep          bool
Cabin_deck       object
Cabin_num         int64
Cabin_side       object
Destination      object
Age             float64
VIP                bool
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Transported        bool
dtype: object

In [14]:
# one hot encoding
train_data = oneHot(train_data, 'train')
test_data = oneHot(test_data, 'test')

train_data.dtypes

HomePlanet_Earth                bool
HomePlanet_Europa               bool
HomePlanet_Mars                 bool
CryoSleep                       bool
Cabin_deck_A                    bool
Cabin_deck_B                    bool
Cabin_deck_C                    bool
Cabin_deck_D                    bool
Cabin_deck_E                    bool
Cabin_deck_F                    bool
Cabin_deck_G                    bool
Cabin_deck_T                    bool
Cabin_num                      int64
Cabin_side_P                    bool
Cabin_side_S                    bool
Destination_55 Cancri e         bool
Destination_PSO J318.5-22       bool
Destination_TRAPPIST-1e         bool
Age                          float64
VIP                             bool
RoomService                  float64
FoodCourt                    float64
ShoppingMall                 float64
Spa                          float64
VRDeck                       float64
Transported                     bool
dtype: object

In [15]:
# normalization
train_data = norm(train_data)
test_data = norm(test_data)

train_data

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep,Cabin_deck_A,Cabin_deck_B,Cabin_deck_C,Cabin_deck_D,Cabin_deck_E,Cabin_deck_F,...,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,False,True,False,False,False,True,False,False,False,False,...,False,True,0.493671,False,0.000000,0.000000,0.000000,0.000000,0.000000,False
1,True,False,False,False,False,False,False,False,False,True,...,False,True,0.303797,False,0.007608,0.000302,0.001064,0.024500,0.001823,True
2,False,True,False,False,True,False,False,False,False,False,...,False,True,0.734177,True,0.003001,0.119948,0.000000,0.299670,0.002030,False
3,False,True,False,False,True,False,False,False,False,False,...,False,True,0.417722,False,0.000000,0.043035,0.015793,0.148563,0.007997,False
4,True,False,False,False,False,False,False,False,False,True,...,False,True,0.202532,False,0.021149,0.002348,0.006428,0.025214,0.000083,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,False,True,False,False,True,False,False,False,False,False,...,False,False,0.518987,True,0.000000,0.228726,0.000000,0.073322,0.003066,False
8689,True,False,False,True,False,False,False,False,False,False,...,True,False,0.227848,False,0.000000,0.000000,0.000000,0.000000,0.000000,False
8690,True,False,False,False,False,False,False,False,False,False,...,False,True,0.329114,False,0.000000,0.000000,0.079687,0.000045,0.000000,True
8691,False,True,False,False,False,False,False,False,True,False,...,False,False,0.405063,False,0.000000,0.035186,0.000000,0.015753,0.134049,False


In [16]:
test_data

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep,Cabin_deck_A,Cabin_deck_B,Cabin_deck_C,Cabin_deck_D,Cabin_deck_E,Cabin_deck_F,...,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,True,False,False,True,False,False,False,False,False,False,...,False,False,True,0.341772,False,0.000000,0.000000,0.00000,0.000000,0.000000
1,True,False,False,False,False,False,False,False,False,True,...,False,False,True,0.240506,False,0.000000,0.000356,0.00000,0.142260,0.000000
2,False,True,False,True,False,False,True,False,False,False,...,True,False,False,0.392405,False,0.000000,0.000000,0.00000,0.000000,0.000000
3,False,True,False,False,False,False,True,False,False,False,...,False,False,True,0.481013,False,0.000000,0.263206,0.00000,0.009121,0.026266
4,True,False,False,False,False,False,False,False,False,True,...,False,False,True,0.253165,False,0.000865,0.000000,0.07658,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,True,False,False,True,False,False,False,False,False,False,...,False,False,True,0.430380,False,0.000000,0.000000,0.00000,0.000000,0.000000
4273,True,False,False,False,False,False,False,False,False,False,...,False,False,True,0.531646,False,0.000000,0.033514,0.00205,0.000504,0.006466
4274,False,False,True,True,False,False,False,True,False,False,...,True,False,False,0.362761,False,0.000000,0.000000,0.00000,0.000000,0.000000
4275,False,True,False,False,False,False,False,True,False,False,...,False,False,False,0.362761,False,0.000000,0.106042,0.00000,0.000000,0.023482


In [18]:
# export preprocessed data into csv
train_export = '../preprocess_train_dataset/normalized_train_data.csv'
test_export = '../preprocess_test_dataset/normalized_test_data.csv'

train_data.to_csv(train_export, sep=',', encoding='utf-8', index=False)
test_data.to_csv(test_export, sep=',', encoding='utf-8', index=False)
